In [1]:
import keras
from keras.models import load_model
from keras.models import model_from_json
import numpy
import os

Using TensorFlow backend.


In [2]:
file_name = 'KERAS_conv1d.json'
json_file = open(file_name, 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("KERAS_conv1d_weights.h5")
print("Loaded model")

ValueError: Unknown constraint: ZeroSomeWeights

In [ ]:
loaded_model.summary()